In [1]:
#! pip install pyspark
#! pip install pyarrow

In [2]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, date_format

import pandas as pd

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1" 
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/27 18:36:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
attend = spark.read.csv('attend.csv', header=True, inferSchema=True)
attend.printSchema()

root
 |-- уникальный номер занятия: integer (nullable = true)
 |-- уникальный номер группы: integer (nullable = true)
 |-- уникальный номер участника: integer (nullable = true)
 |-- направление 2: string (nullable = true)
 |-- направление 3: string (nullable = true)
 |-- онлайн/офлайн: string (nullable = true)
 |-- дата занятия: date (nullable = true)
 |-- время начала занятия: timestamp (nullable = true)
 |-- время окончания занятия: timestamp (nullable = true)



In [4]:
attend = attend.withColumn("дата занятия",to_timestamp(col("дата занятия"))).withColumn("day_of_week", date_format(col("дата занятия"), "EEEE"))
attend.show()

+------------------------+-----------------------+--------------------------+--------------------+--------------------+-------------+-------------------+--------------------+-----------------------+-----------+
|уникальный номер занятия|уникальный номер группы|уникальный номер участника|       направление 2|       направление 3|онлайн/офлайн|       дата занятия|время начала занятия|время окончания занятия|day_of_week|
+------------------------+-----------------------+--------------------------+--------------------+--------------------+-------------+-------------------+--------------------+-----------------------+-----------+
|               401346550|              801346550|                 101352023|   ОНЛАЙН Гимнастика|        ОНЛАЙН Цигун|           Да|2022-08-01 00:00:00| 2023-05-27 09:00:00|    2023-05-27 10:00:00|     Monday|
|               401346550|              801346550|                 101385462|   ОНЛАЙН Гимнастика|        ОНЛАЙН Цигун|           Да|2022-08-01 00:00:00| 20

In [5]:
attend = attend.withColumn("дата занятия",to_timestamp(col("дата занятия"))).withColumn("is_sunday", col("day_of_week")=="Sunday")
attend = attend.withColumn("дата занятия",to_timestamp(col("дата занятия"))).withColumn("is_saturday", col("day_of_week")=="Saturday")
attend = attend.withColumn("дата занятия",to_timestamp(col("дата занятия"))).withColumn("is_weekend", col("is_saturday")|col("is_sunday"))
attend = attend.drop("is_saturday","is_sunday")
attend.show()

+------------------------+-----------------------+--------------------------+--------------------+--------------------+-------------+-------------------+--------------------+-----------------------+-----------+----------+
|уникальный номер занятия|уникальный номер группы|уникальный номер участника|       направление 2|       направление 3|онлайн/офлайн|       дата занятия|время начала занятия|время окончания занятия|day_of_week|is_weekend|
+------------------------+-----------------------+--------------------------+--------------------+--------------------+-------------+-------------------+--------------------+-----------------------+-----------+----------+
|               401346550|              801346550|                 101352023|   ОНЛАЙН Гимнастика|        ОНЛАЙН Цигун|           Да|2022-08-01 00:00:00| 2023-05-27 09:00:00|    2023-05-27 10:00:00|     Monday|     false|
|               401346550|              801346550|                 101385462|   ОНЛАЙН Гимнастика|        ОНЛАЙН

In [6]:
attend.repartition(1).write.csv("is_weekend_feature_added.csv", )

In [7]:
df = pd.read_csv("is_weekend_feature_added.csv")
df.head()

,401346550,801346550,101352023,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01T00:00:00.000+03:00,2023-05-27T09:00:00.000+03:00,2023-05-27T10:00:00.000+03:00,Monday,false
0,401346550,801346550,101385462,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01T00:00:00.000+03:00,2023-05-27T09:00:00.000+03:00,2023-05-27T10:00:00.000+03:00,Monday,False
1,401346550,801346550,101421897,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01T00:00:00.000+03:00,2023-05-27T09:00:00.000+03:00,2023-05-27T10:00:00.000+03:00,Monday,False
2,401346550,801346550,101354499,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01T00:00:00.000+03:00,2023-05-27T09:00:00.000+03:00,2023-05-27T10:00:00.000+03:00,Monday,False
3,401346550,801346550,101421312,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01T00:00:00.000+03:00,2023-05-27T09:00:00.000+03:00,2023-05-27T10:00:00.000+03:00,Monday,False
4,401346550,801346550,101417874,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01T00:00:00.000+03:00,2023-05-27T09:00:00.000+03:00,2023-05-27T10:00:00.000+03:00,Monday,False


In [9]:
del df